In [41]:
import numpy as np
import h5py
import pandas as pd

In [42]:
#Leer datos de .mat files y guardarlos en un numpy array
data = {}
data_aoi = {}
targets = {}
targets_aoi = {}
files = open('files_polygon', 'r')
lines = files.readlines()
count = {}
count_aoi = {}
aois = []
for line in lines:
    fileName, target, aoi = line.strip().split(";")
    aois.append(aoi)
    with h5py.File(fileName, 'r') as f:
        for k, v in f.items():
            tmp=np.array(v).transpose()
            if not target in data.keys():
                data[target]=tmp
                count[target] = tmp.shape[0]
            else:
                data[target] = np.concatenate((data[target], tmp))
                count[target]+= tmp.shape[0]
            if not aoi in data_aoi.keys():
                data_aoi[aoi]=tmp.shape[0]
aoiColumn = None
i=0
for k, v in data_aoi.items():
    targets_aoi[k]=np.ones(v, dtype=int)*int(i)
    if i==0:
        aoiColumn = targets_aoi[k]
    else:
        aoiColumn = np.concatenate((aoiColumn, targets_aoi[k]))
    i+=1
aoiColumn = aoiColumn.reshape((aoiColumn.shape[0]), 1)
totalCount = 0
i=0
targetsColumn = None
for k, v in count.items():
    targets[k]=np.ones(v, dtype=int)*int(i)
    totalCount += v
    if i==0:
        targetsColumn = targets[k]
    else:
        targetsColumn = np.concatenate((targetsColumn, targets[k]))
    i += 1
targetsColumn=targetsColumn.reshape(targetsColumn.shape[0], 1)
dataSet = None
for k, v in data.items():
    if dataSet is None:
        dataSet = data[k]
    else:
        dataSet = np.concatenate((dataSet, data[k]))
dataSet = np.concatenate((dataSet, aoiColumn), axis=1)
dataSet = np.concatenate((dataSet, targetsColumn), axis=1)
print(aois)

['caqueta_20160311', 'guaviare_20160125', 'guaviare_20160820', 'putumayo_20160311', 'nubes_20160428', 'nubes_20170226', 'nubes_20190509', 'cauca_20190818', 'cauca_20190903', 'caqueta_bosque_20150221', 'caqueta_bosque_20160311', 'guaviare_bosque_20150207', 'guaviare_bosque_20160125', 'guaviare_bosque_20160820', 'guaviare_bosque_20171010', 'putumayo_bosque_20160311']


In [43]:
from scipy import stats

#aoi = ["caqueta_20160311", "guaviare_20160125", "guaviare_20160820", "putumayo_20160311", "nubes_20160428", 
#      "nubes_20170226", "nubes_20190509", "caqueta_bosque_20160311", "guaviare_bosque_20160125", "putumayo_bosque_20160311",
#       "cauca_20190818", "cauca_20190903"]

aoi = aois

df = pd.DataFrame(data=dataSet, columns=["Costal", "Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2", "Longitude", "Latitude", "x", "y", "AOI","Target" ])
df=df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]
df['Target'] = df['Target'].replace({0:"coca", 1:"nube", 2:"cana", 3:"bosque"})
df['AOI'] = df['AOI'].replace({0:aoi[0], 1:aoi[1], 2:aoi[2], 3:aoi[3], 4:aoi[4], 5:aoi[5],
                               6:aoi[6], 7:aoi[7], 8:aoi[8], 9:aoi[9], 10:aoi[10], 11:aoi[11],
                              12:aoi[12], 13:aoi[13], 14:aoi[14], 15:aoi[15]})
df


,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target
0,10856.0,9761.0,8767.0,7646.0,17400.0,11280.0,7453.0,-75.139976,0.609742,7196.0,13868.0,caqueta_20160311,coca
1,10883.0,9767.0,8845.0,7616.0,17759.0,10465.0,7143.0,-75.142268,0.614220,7179.0,13835.0,caqueta_20160311,coca
2,10893.0,9787.0,8797.0,7574.0,17908.0,10563.0,7026.0,-75.354861,0.635109,5602.0,13681.0,caqueta_20160311,coca
3,10844.0,9813.0,8808.0,7617.0,17533.0,10779.0,7173.0,-75.130945,0.639055,7263.0,13652.0,caqueta_20160311,coca
4,10945.0,9848.0,9001.0,7699.0,21179.0,12428.0,7952.0,-75.300805,0.660083,6003.0,13497.0,caqueta_20160311,coca
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53161,11902.0,10836.0,10014.0,8663.0,25653.0,15408.0,9496.0,-75.481014,0.242369,6946.0,5935.0,putumayo_bosque_20160311,bosque
53162,11656.0,10548.0,9551.0,8482.0,13847.0,8920.0,6931.0,-74.765087,0.110738,12257.0,6905.0,putumayo_bosque_20160311,bosque
53163,11748.0,10729.0,9596.0,8584.0,19133.0,11754.0,7299.0,-74.604946,-0.021035,13445.0,7876.0,putumayo_bosque_20160311,bosque
53164,11839.0,10658.0,9601.0,8374.0,19394.0,12201.0,7884.0,-76.020085,0.590243,2947.0,3371.0,putumayo_bosque_20160311,bosque


In [44]:
cocaCond = df['Target']=='coca'
cocaDF = df[cocaCond]
cocaDF

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target
0,10856.0,9761.0,8767.0,7646.0,17400.0,11280.0,7453.0,-75.139976,0.609742,7196.0,13868.0,caqueta_20160311,coca
1,10883.0,9767.0,8845.0,7616.0,17759.0,10465.0,7143.0,-75.142268,0.614220,7179.0,13835.0,caqueta_20160311,coca
2,10893.0,9787.0,8797.0,7574.0,17908.0,10563.0,7026.0,-75.354861,0.635109,5602.0,13681.0,caqueta_20160311,coca
3,10844.0,9813.0,8808.0,7617.0,17533.0,10779.0,7173.0,-75.130945,0.639055,7263.0,13652.0,caqueta_20160311,coca
4,10945.0,9848.0,9001.0,7699.0,21179.0,12428.0,7952.0,-75.300805,0.660083,6003.0,13497.0,caqueta_20160311,coca
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16550,12570.0,11499.0,10301.0,9135.0,19587.0,11664.0,7920.0,-75.471487,0.814637,7017.0,1718.0,putumayo_20160311,coca
16551,12910.0,11847.0,10507.0,9415.0,18205.0,11352.0,8083.0,-75.313221,0.847494,8191.0,1476.0,putumayo_20160311,coca
16552,13641.0,12630.0,11277.0,10241.0,18879.0,11882.0,8490.0,-75.327646,0.852378,8084.0,1440.0,putumayo_20160311,coca
16553,12006.0,10930.0,9883.0,8506.0,22377.0,12739.0,7981.0,-75.119493,0.879668,9628.0,1239.0,putumayo_20160311,coca


In [45]:
nubeCond = df['Target']=='nube'
nubeDF = df[nubeCond]
nubeDF

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target
16555,45525.0,45270.0,43150.0,44009.0,45064.0,13799.0,15248.0,1.0,1.0,0.0,0.0,nubes_20160428,nube
16556,45494.0,45131.0,43098.0,43843.0,44881.0,13794.0,15348.0,1.0,1.0,0.0,0.0,nubes_20160428,nube
16557,45829.0,45244.0,43380.0,44218.0,45294.0,13803.0,15270.0,1.0,1.0,0.0,0.0,nubes_20160428,nube
16558,48256.0,47760.0,45782.0,46627.0,47720.0,13469.0,15103.0,1.0,1.0,0.0,0.0,nubes_20160428,nube
16559,48294.0,47869.0,45748.0,46651.0,47732.0,13450.0,15110.0,1.0,1.0,0.0,0.0,nubes_20160428,nube
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27094,34738.0,34379.0,32773.0,33166.0,35492.0,11176.0,12439.0,1.0,1.0,0.0,0.0,nubes_20190509,nube
27095,36910.0,36711.0,35047.0,35554.0,37579.0,11245.0,12491.0,1.0,1.0,0.0,0.0,nubes_20190509,nube
27096,37035.0,36864.0,35106.0,35708.0,37718.0,11377.0,12672.0,1.0,1.0,0.0,0.0,nubes_20190509,nube
27097,39316.0,39171.0,37337.0,37978.0,40041.0,10979.0,12077.0,1.0,1.0,0.0,0.0,nubes_20190509,nube


In [46]:
bosqueCond = df['Target']=='bosque'
bosqueDF = df[bosqueCond]
bosqueDF

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target
44081,10079.0,9140.0,9048.0,7389.0,22689.0,12272.0,7893.0,-75.319012,0.773668,5868.0,12660.0,caqueta_bosque_20150221,bosque
44082,10193.0,9138.0,8378.0,7217.0,18260.0,10609.0,7023.0,-75.736412,0.963186,2772.0,11263.0,caqueta_bosque_20150221,bosque
44083,10320.0,9362.0,8896.0,8017.0,18176.0,14126.0,9248.0,-75.731966,0.973636,2805.0,11186.0,caqueta_bosque_20150221,bosque
44084,10311.0,9347.0,9053.0,7802.0,19765.0,15062.0,9483.0,-75.830107,0.977006,2077.0,11161.0,caqueta_bosque_20150221,bosque
44085,10323.0,9339.0,9229.0,7834.0,21197.0,13565.0,8702.0,-75.759200,0.986657,2603.0,11090.0,caqueta_bosque_20150221,bosque
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53161,11902.0,10836.0,10014.0,8663.0,25653.0,15408.0,9496.0,-75.481014,0.242369,6946.0,5935.0,putumayo_bosque_20160311,bosque
53162,11656.0,10548.0,9551.0,8482.0,13847.0,8920.0,6931.0,-74.765087,0.110738,12257.0,6905.0,putumayo_bosque_20160311,bosque
53163,11748.0,10729.0,9596.0,8584.0,19133.0,11754.0,7299.0,-74.604946,-0.021035,13445.0,7876.0,putumayo_bosque_20160311,bosque
53164,11839.0,10658.0,9601.0,8374.0,19394.0,12201.0,7884.0,-76.020085,0.590243,2947.0,3371.0,putumayo_bosque_20160311,bosque


In [47]:
canaCond = df['Target']=='cana'
canaDF = df[canaCond]
canaDF

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target
27103,10793.0,10004.0,9701.0,8932.0,16629.0,12424.0,9200.0,-76.599301,3.018602,5465.0,6789.0,cauca_20190818,cana
27104,10335.0,9521.0,8967.0,8022.0,18219.0,12143.0,8506.0,-76.592424,3.021732,5516.0,6766.0,cauca_20190818,cana
27105,10217.0,9280.0,8983.0,7712.0,20643.0,13086.0,8604.0,-76.589056,3.025671,5541.0,6737.0,cauca_20190818,cana
27106,10474.0,9660.0,9438.0,8745.0,14371.0,12234.0,9198.0,-76.581099,3.028532,5600.0,6716.0,cauca_20190818,cana
27107,10778.0,9946.0,9086.0,8858.0,13159.0,8852.0,7200.0,-76.582187,3.034499,5592.0,6672.0,cauca_20190818,cana
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44076,10053.0,9231.0,8983.0,8140.0,21430.0,13972.0,9350.0,-76.357971,3.289182,7216.0,4797.0,cauca_20190903,cana
44077,9465.0,8638.0,8662.0,8334.0,21553.0,12664.0,7129.0,-76.355002,3.289864,7238.0,4792.0,cauca_20190903,cana
44078,10026.0,9285.0,9514.0,8931.0,21104.0,14579.0,8932.0,-76.354462,3.290000,7242.0,4791.0,cauca_20190903,cana
44079,10004.0,9281.0,9451.0,8706.0,23822.0,16861.0,9970.0,-76.437893,3.298023,6624.0,4731.0,cauca_20190903,cana


In [48]:
df=df.drop_duplicates(subset=['Longitude', 'Latitude'])
df=pd.concat([df, nubeDF])
df

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target
0,10856.0,9761.0,8767.0,7646.0,17400.0,11280.0,7453.0,-75.139976,0.609742,7196.0,13868.0,caqueta_20160311,coca
1,10883.0,9767.0,8845.0,7616.0,17759.0,10465.0,7143.0,-75.142268,0.614220,7179.0,13835.0,caqueta_20160311,coca
2,10893.0,9787.0,8797.0,7574.0,17908.0,10563.0,7026.0,-75.354861,0.635109,5602.0,13681.0,caqueta_20160311,coca
3,10844.0,9813.0,8808.0,7617.0,17533.0,10779.0,7173.0,-75.130945,0.639055,7263.0,13652.0,caqueta_20160311,coca
4,10945.0,9848.0,9001.0,7699.0,21179.0,12428.0,7952.0,-75.300805,0.660083,6003.0,13497.0,caqueta_20160311,coca
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27094,34738.0,34379.0,32773.0,33166.0,35492.0,11176.0,12439.0,1.000000,1.000000,0.0,0.0,nubes_20190509,nube
27095,36910.0,36711.0,35047.0,35554.0,37579.0,11245.0,12491.0,1.000000,1.000000,0.0,0.0,nubes_20190509,nube
27096,37035.0,36864.0,35106.0,35708.0,37718.0,11377.0,12672.0,1.000000,1.000000,0.0,0.0,nubes_20190509,nube
27097,39316.0,39171.0,37337.0,37978.0,40041.0,10979.0,12077.0,1.000000,1.000000,0.0,0.0,nubes_20190509,nube


In [49]:
bosqueCond = df['Target']=='bosque'
bosqueDF = df[bosqueCond]
bosqueDF

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target
44081,10079.0,9140.0,9048.0,7389.0,22689.0,12272.0,7893.0,-75.319012,0.773668,5868.0,12660.0,caqueta_bosque_20150221,bosque
44082,10193.0,9138.0,8378.0,7217.0,18260.0,10609.0,7023.0,-75.736412,0.963186,2772.0,11263.0,caqueta_bosque_20150221,bosque
44083,10320.0,9362.0,8896.0,8017.0,18176.0,14126.0,9248.0,-75.731966,0.973636,2805.0,11186.0,caqueta_bosque_20150221,bosque
44084,10311.0,9347.0,9053.0,7802.0,19765.0,15062.0,9483.0,-75.830107,0.977006,2077.0,11161.0,caqueta_bosque_20150221,bosque
44085,10323.0,9339.0,9229.0,7834.0,21197.0,13565.0,8702.0,-75.759200,0.986657,2603.0,11090.0,caqueta_bosque_20150221,bosque
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53161,11902.0,10836.0,10014.0,8663.0,25653.0,15408.0,9496.0,-75.481014,0.242369,6946.0,5935.0,putumayo_bosque_20160311,bosque
53162,11656.0,10548.0,9551.0,8482.0,13847.0,8920.0,6931.0,-74.765087,0.110738,12257.0,6905.0,putumayo_bosque_20160311,bosque
53163,11748.0,10729.0,9596.0,8584.0,19133.0,11754.0,7299.0,-74.604946,-0.021035,13445.0,7876.0,putumayo_bosque_20160311,bosque
53164,11839.0,10658.0,9601.0,8374.0,19394.0,12201.0,7884.0,-76.020085,0.590243,2947.0,3371.0,putumayo_bosque_20160311,bosque


In [50]:
#Balancing the dataframe

coca = cocaDF.sample(n=4122)
nube = nubeDF.sample(n=4122)
cana = canaDF.sample(n=4122)
bosque = bosqueDF.sample(n=4122)
print(coca.shape, nube.shape, cana.shape, bosque.shape)

df = pd.concat([coca, nube, cana, bosque])
df

(4122, 13) (4122, 13) (4122, 13) (4122, 13)


,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target
15614,11889.0,10831.0,9776.0,9061.0,15070.0,12856.0,9986.0,-76.140695,0.471497,2052.0,4246.0,putumayo_20160311,coca
1597,11193.0,10141.0,9546.0,8500.0,19655.0,14056.0,9199.0,-75.114377,1.086760,7386.0,10353.0,caqueta_20160311,coca
8488,10320.0,9271.0,8350.0,7269.0,17828.0,10559.0,7132.0,-72.080592,1.982562,6948.0,3753.0,guaviare_20160125,coca
5755,11107.0,10096.0,9440.0,8510.0,17608.0,15952.0,10923.0,-74.567905,1.272104,11439.0,8987.0,caqueta_20160311,coca
14150,11500.0,10392.0,9380.0,8194.0,18888.0,12022.0,7919.0,-74.865108,0.256762,11515.0,5829.0,putumayo_20160311,coca
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44799,10416.0,9512.0,9179.0,8494.0,18914.0,13364.0,9041.0,-75.671689,0.886801,3252.0,11826.0,caqueta_bosque_20150221,bosque
45388,10210.0,9158.0,8427.0,7282.0,17519.0,10518.0,7307.0,-75.430254,0.980474,5043.0,11136.0,caqueta_bosque_20150221,bosque
52907,12076.0,10991.0,9885.0,8665.0,19521.0,12419.0,8152.0,-75.559642,0.737411,6363.0,2287.0,putumayo_bosque_20160311,bosque
49306,12607.0,11414.0,11773.0,10749.0,22374.0,16045.0,11900.0,-71.821121,1.792790,8877.0,5150.0,guaviare_bosque_20150207,bosque


In [51]:
#Shuffle dataset mix
df = df.sample(frac=1).reset_index(drop=True)
df

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target
0,41155.0,40919.0,38807.0,39495.0,41310.0,17609.0,16032.0,1.000000,1.000000,0.0,0.0,nubes_20160428,nube
1,11078.0,9979.0,9120.0,7769.0,20546.0,12158.0,7493.0,-75.831336,0.112627,4347.0,6891.0,putumayo_bosque_20160311,bosque
2,44577.0,44441.0,42663.0,43377.0,44969.0,17264.0,15534.0,1.000000,1.000000,0.0,0.0,nubes_20160428,nube
3,10628.0,9630.0,8682.0,7635.0,18300.0,11348.0,7559.0,-72.094163,2.220039,6844.0,2001.0,guaviare_20160125,coca
4,11232.0,10146.0,9511.0,8343.0,18440.0,12152.0,8177.0,-75.335603,0.886033,5745.0,11832.0,caqueta_20160311,coca
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16483,41899.0,41875.0,39801.0,40555.0,41173.0,11958.0,13295.0,1.000000,1.000000,0.0,0.0,nubes_20170226,nube
16484,12545.0,11459.0,10205.0,9110.0,18460.0,11283.0,7870.0,-75.390192,0.768913,7620.0,2055.0,putumayo_20160311,coca
16485,10359.0,9690.0,10101.0,8905.0,22012.0,14013.0,8944.0,-76.450033,3.197887,6533.0,5469.0,cauca_20190903,cana
16486,39599.0,39292.0,37538.0,38229.0,39376.0,12335.0,13495.0,1.000000,1.000000,0.0,0.0,nubes_20190509,nube


In [6]:
#Creating the sub sets : Training set 70%, Test set 20%, Validation set 10%

In [52]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

features = ["Costal", "Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2"]

labels = ["coca","nube", "cana", "bosque"]

X=df.loc[:, features]
y=df.loc[:, "Target"]

"""
min_max_scaler = preprocessing.MinMaxScaler()
Xp=min_max_scaler.fit_transform(X)
Xp = pd.DataFrame(Xp)
X = pd.concat([X, Xp], axis=1)
"""

print(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109)

        Costal     Blue    Green      Red      NIR    SWIR1    SWIR2
0      41155.0  40919.0  38807.0  39495.0  41310.0  17609.0  16032.0
1      11078.0   9979.0   9120.0   7769.0  20546.0  12158.0   7493.0
2      44577.0  44441.0  42663.0  43377.0  44969.0  17264.0  15534.0
3      10628.0   9630.0   8682.0   7635.0  18300.0  11348.0   7559.0
4      11232.0  10146.0   9511.0   8343.0  18440.0  12152.0   8177.0
...        ...      ...      ...      ...      ...      ...      ...
16483  41899.0  41875.0  39801.0  40555.0  41173.0  11958.0  13295.0
16484  12545.0  11459.0  10205.0   9110.0  18460.0  11283.0   7870.0
16485  10359.0   9690.0  10101.0   8905.0  22012.0  14013.0   8944.0
16486  39599.0  39292.0  37538.0  38229.0  39376.0  12335.0  13495.0
16487  35607.0  35247.0  33618.0  33937.0  36901.0  19480.0  17395.0

[16488 rows x 7 columns]


In [122]:
#Fitting parameters and validating it with validation set
#from sklearn import svm
#from sklearn import tree
#from sklearn.preprocessing import StandardScaler
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNB
#from sklearn.ensemble import BaggingClassifier

vdf = df.sample(frac=0.15).reset_index(drop=True)

X_v = vdf.loc[:, features]
y_v = vdf.loc[:, "Target"]

X_v_train, X_v_test, y_v_train, y_v_test = train_test_split(X_v, y_v, test_size=0.3, random_state=109)

#classifier = svm.SVC(kernel='linear', class_weight='balanced')
#classifier = tree.DecisionTreeClassifier()
#classifier = RandomForestClassifier(n_estimators=10)
classifier = KNeighborsClassifier()
#classifier = BaggingClassifier(base_estimator=SVC(), n_estimators=10, random_state=0)

#scaler = StandardScaler(with_mean=False, with_std=False)#MinMaxScaler()
#X_v_train = scaler.fit_transform(X_v_train)
#X_v_test = scaler.fit_transform(X_v_test)
    
#Normalizar z score, z normalization,

classifier.fit(X_v_train, y_v_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [123]:
from sklearn import metrics

y_v_pred = classifier.predict(X_v_test)

cf=metrics.confusion_matrix(y_v_test, y_v_pred, labels)
cf

array([[128,   0,   9,  39],
       [  0, 203,   0,   0],
       [ 10,   0, 117,  33],
       [ 71,   0,  17, 115]])

In [124]:
print(metrics.accuracy_score(y_v_test, y_v_pred, labels))

rows = [0 for _ in range(len(cf))]
percents = [[] for _ in range(len(cf))]
for i in range(len(cf)):
    for j in range(len(cf[i])):
        rows[i]+=cf[i][j]
for i in range(len(cf)):
    for j in range(len(cf[i])):
        if i==j:
            percents[i].append(cf[i][j]/rows[i])
print(percents)


0.7587601078167115
[[0.7272727272727273], [1.0], [0.73125], [0.5665024630541872]]


In [38]:
#Calculating mean of each band per each class
features = ["Costal", "Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2"]

coca = coca.loc[:, features]
nube = nube.loc[:, features]
cana = cana.loc[:, features]
bosque = bosque.loc[:, features]

coca_mean=coca.mean()
nube_mean=nube.mean()
cana_mean=cana.mean()
bosque_mean=bosque.mean()

coca_var = coca.var()
nube_var = nube.var()
cana_var = cana.var()
bosque_var = bosque.var()

In [39]:
from scipy.stats import entropy
from math import log, e

def pandas_entropy(column, base=None):
  vc = pd.Series(column).value_counts(normalize=True, sort=False)
  base = e if base is None else base
  return -(vc * np.log(vc)/np.log(base)).sum()

bands = len(features)
features = ["Costal", "Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2"]
labels = ['coca', 'nube', 'bosque', 'cana']
cluster = [coca, nube, cana, bosque]

values = [list() for _ in cluster]

for i in range(len(cluster)):
    for j in range(len(features)):
        values[i].append(pandas_entropy(cluster[i][features[j]]))

coca_entropy = pd.DataFrame(values[0], index=features)
nube_entropy = pd.DataFrame(values[1], index=features)
bosque_entropy = pd.DataFrame(values[2], index=features)
cana_entropy = pd.DataFrame(values[3], index=features)

In [37]:
from mpl_toolkits import mplot3d

%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(11,11))
ax = plt.axes(projection='3d', label='coca', markersize=10)

cM = coca_mean.to_numpy().tolist()
cV = coca_var.to_numpy().tolist()
cE = coca_entropy[0].to_numpy().tolist()

ax.scatter(cM, cV, cE, label='coca', color='cyan', marker='^')

nM = nube_mean.to_numpy().tolist()
nV = nube_var.to_numpy().tolist()
nE = nube_entropy[0].to_numpy().tolist()

ax.scatter(nM, nV, nE, color='red', label='nube', marker='s')

caM = cana_mean.to_numpy().tolist()
caV = cana_var.to_numpy().tolist()
caE = cana_entropy[0].to_numpy().tolist()

ax.scatter(caM, caV, caE, color='green', label='caña', marker='o')

bM = bosque_mean.to_numpy().tolist()
bV = bosque_var.to_numpy().tolist()
bE = bosque_entropy[0].to_numpy().tolist()

ax.scatter(bM, bV, bE, color='yellow', label='bosque', marker='*')
ax.legend()

AttributeError: 'Axes3DSubplot' object has no property 'markersize'

<Figure size 792x792 with 0 Axes>

In [37]:
#Saving the model to a file
import pickle
filename = './Classifiers/validation_classifier.sav'
pickle.dump(classifier, open(filename, 'wb'))
#loaded_model = pickle.load(open(filename, 'rb'))

In [427]:
import pickle
filename = './Classifiers/validation_classifier.sav'
classifier = pickle.load(open(filename, 'rb'))

In [428]:
XX=vdf.loc[:, features]
yy=classifier.predict(XX)

In [429]:
from sklearn import metrics
print(metrics.confusion_matrix(y_v, yy, labels))
print(metrics.precision_recall_fscore_support(y_v, yy))

[[1335    0   65 1131]
 [   0 1525    0    0]
 [  33    0 2488   50]
 [ 688    1   78 1399]]
(array([0.54224806, 0.94564804, 0.64931907, 0.99934469]), array([0.64589104, 0.96771684, 0.5274595 , 1.        ]), array([0.58954909, 0.95655517, 0.58207979, 0.99967224]), array([2166, 2571, 2531, 1525]))


In [430]:
from sklearn import metrics
print("Accuracy: ", metrics.accuracy_score(y_v_test, y_v_pred))
#The classifier works fine on validation set, now it's time to training over all the data set and
#Check if results are the same

Accuracy:  0.7649734647460197


In [53]:
X_train #70% of all the data sample

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2
5971,13011.0,12212.0,11385.0,11208.0,16166.0,12466.0,9937.0
2099,11197.0,10144.0,9474.0,8525.0,17697.0,14239.0,9538.0
3708,9585.0,8498.0,7877.0,6597.0,19792.0,12103.0,7422.0
12112,10197.0,9099.0,8346.0,7181.0,21416.0,12920.0,8075.0
13724,9702.0,8863.0,8688.0,7945.0,18140.0,14012.0,9460.0
...,...,...,...,...,...,...,...
298,11653.0,10844.0,10639.0,10700.0,18637.0,18509.0,13260.0
3873,10383.0,9515.0,8908.0,8812.0,15275.0,13887.0,9653.0
16368,41290.0,41068.0,39332.0,40067.0,41528.0,12127.0,13368.0
7925,45890.0,45685.0,44092.0,44822.0,44762.0,13202.0,15184.0


In [54]:
#Clasificador para los datos inmensos
from sklearn import svm
classifier = svm.SVC(kernel='linear', class_weight='balanced')
classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [57]:
from sklearn import metrics

y_pred = classifier.predict(X_test)

cf=metrics.confusion_matrix(y_test, y_pred, labels)
cf

array([[ 801,    0,   23,  394],
       [   0, 1219,    0,    0],
       [   3,    0, 1165,   80],
       [ 374,    0,  102,  786]])

In [59]:
#Prediction and metrics of the classifier predicts to the entire test set
print(metrics.accuracy_score(y_test, y_pred, labels))

rows = [0 for _ in range(len(cf))]
percents = [[] for _ in range(len(cf))]
for i in range(len(cf)):
    for j in range(len(cf[i])):
        rows[i]+=cf[i][j]
for i in range(len(cf)):
    for j in range(len(cf[i])):
        if i==j:
            percents[i].append(cf[i][j]/rows[i])
print(percents)

0.8027087123509198
[[0.6576354679802956], [1.0], [0.9334935897435898], [0.6228209191759112]]


In [61]:
#Saving the model to a file
import pickle
filename = './Classifiers/full_svm_classifier.sav'
pickle.dump(classifier, open(filename, 'wb'))
#loaded_model = pickle.load(open(filename, 'rb'))

In [109]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=10)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [110]:
from sklearn import metrics

y_pred = classifier.predict(X_test)

cf=metrics.confusion_matrix(y_test, y_pred, labels)
cf

array([[ 883,    0,   33,  302],
       [   0, 1219,    0,    0],
       [  16,    0, 1137,   95],
       [ 233,    2,  123,  904]])

In [111]:
print(metrics.accuracy_score(y_test, y_pred, labels))

rows = [0 for _ in range(len(cf))]
percents = [[] for _ in range(len(cf))]
for i in range(len(cf)):
    for j in range(len(cf[i])):
        rows[i]+=cf[i][j]
for i in range(len(cf)):
    for j in range(len(cf[i])):
        if i==j:
            percents[i].append(cf[i][j]/rows[i])
print(percents)

0.8374772589448151
[[0.7249589490968801], [1.0], [0.9110576923076923], [0.716323296354992]]


In [112]:
#Saving the model to a file
import pickle
filename = './Classifiers/full_rf_classifier.sav'
pickle.dump(classifier, open(filename, 'wb'))
#loaded_model = pickle.load(open(filename, 'rb'))

In [97]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [98]:
from sklearn import metrics

y_pred = classifier.predict(X_test)

cf=metrics.confusion_matrix(y_test, y_pred, labels)
cf

array([[ 911,    0,   47,  260],
       [   0, 1219,    0,    0],
       [  33,    0, 1048,  167],
       [ 339,    2,  133,  788]])

In [99]:
print(metrics.accuracy_score(y_test, y_pred, labels))

rows = [0 for _ in range(len(cf))]
percents = [[] for _ in range(len(cf))]
for i in range(len(cf)):
    for j in range(len(cf[i])):
        rows[i]+=cf[i][j]
for i in range(len(cf)):
    for j in range(len(cf[i])):
        if i==j:
            percents[i].append(cf[i][j]/rows[i])
print(percents)

0.8016979987871438
[[0.7479474548440066], [1.0], [0.8397435897435898], [0.624405705229794]]


In [100]:
#Saving the model to a file
import pickle
filename = './Classifiers/full_kn_classifier.sav'
pickle.dump(classifier, open(filename, 'wb'))
#loaded_model = pickle.load(open(filename, 'rb'))

In [113]:
import pickle
filename = './Classifiers/full_rf_classifier.sav'
classifier = None
classifier = pickle.load(open(filename, 'rb'))

In [114]:
big_pred = classifier.predict(X)
print("Accuracy Big Prediction:", metrics.accuracy_score(y, big_pred))
cf = metrics.confusion_matrix(y, big_pred, labels)
print("Confusion Matrix:\n", cf, '\n')
rows = [0 for _ in range(len(cf))]
percents = [[] for _ in range(len(cf))]
for i in range(len(cf)):
    for j in range(len(cf[i])):
        rows[i]+=cf[i][j]
for i in range(len(cf)):
    for j in range(len(cf[i])):
        if i==j:
            percents[i].append(cf[i][j]/rows[i])
print(percents)

Accuracy Big Prediction: 0.9450509461426492
Confusion Matrix:
 [[3732    1   39  350]
 [   0 4122    0    0]
 [  16    0 4001  105]
 [ 254    2  139 3727]] 

[[0.9053857350800583], [1.0], [0.9706453178068899], [0.9041727316836488]]


In [77]:
metadata_df = df
metadata_df['Prediction']=big_pred
metadata_df

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target,Prediction
0,41155.0,40919.0,38807.0,39495.0,41310.0,17609.0,16032.0,1.000000,1.000000,0.0,0.0,nubes_20160428,nube,nube
1,11078.0,9979.0,9120.0,7769.0,20546.0,12158.0,7493.0,-75.831336,0.112627,4347.0,6891.0,putumayo_bosque_20160311,bosque,coca
2,44577.0,44441.0,42663.0,43377.0,44969.0,17264.0,15534.0,1.000000,1.000000,0.0,0.0,nubes_20160428,nube,nube
3,10628.0,9630.0,8682.0,7635.0,18300.0,11348.0,7559.0,-72.094163,2.220039,6844.0,2001.0,guaviare_20160125,coca,coca
4,11232.0,10146.0,9511.0,8343.0,18440.0,12152.0,8177.0,-75.335603,0.886033,5745.0,11832.0,caqueta_20160311,coca,coca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16483,41899.0,41875.0,39801.0,40555.0,41173.0,11958.0,13295.0,1.000000,1.000000,0.0,0.0,nubes_20170226,nube,nube
16484,12545.0,11459.0,10205.0,9110.0,18460.0,11283.0,7870.0,-75.390192,0.768913,7620.0,2055.0,putumayo_20160311,coca,bosque
16485,10359.0,9690.0,10101.0,8905.0,22012.0,14013.0,8944.0,-76.450033,3.197887,6533.0,5469.0,cauca_20190903,cana,cana
16486,39599.0,39292.0,37538.0,38229.0,39376.0,12335.0,13495.0,1.000000,1.000000,0.0,0.0,nubes_20190509,nube,nube


In [438]:
#Saving some data with predictions to work with the same data
import pickle
pickle.dump(metadata_df, open('./dataPy/metadata_pred.sav', 'wb'))

In [439]:
import pickle
metadata_df = pickle.load(open('./dataPy/metadata_pred.sav', 'rb'))
metadata_df

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target,Prediction
0,9877.0,8929.0,8287.0,7277.0,18061.0,11682.0,7962.0,-76.440268,3.163849,6605.0,5720.0,cauca_20190903,cana,cana
1,35746.0,35471.0,33748.0,34229.0,35780.0,11479.0,12576.0,1.000000,1.000000,0.0,0.0,nubes_20170226,nube,nube
2,11827.0,10733.0,9965.0,8513.0,21257.0,13240.0,8494.0,-75.227987,1.932755,6544.0,4119.0,caqueta_20160311,coca,bosque
3,11389.0,10477.0,10088.0,9139.0,21665.0,17201.0,11071.0,-75.825280,1.080407,2113.0,10399.0,caqueta_bosque_20160311,bosque,bosque
4,41486.0,41279.0,39488.0,40271.0,41467.0,10777.0,11774.0,1.000000,1.000000,0.0,0.0,nubes_20170226,nube,nube
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58614,35613.0,35331.0,33823.0,34285.0,35955.0,11964.0,13060.0,1.000000,1.000000,0.0,0.0,nubes_20190509,nube,nube
58615,10404.0,9358.0,8521.0,7486.0,15883.0,9997.0,7062.0,-71.987101,1.000008,7652.0,11002.0,guaviare_20160125,coca,coca
58616,10168.0,9124.0,8259.0,7250.0,16060.0,9574.0,6872.0,-72.034790,1.240475,7296.0,9228.0,guaviare_bosque_20160125,bosque,coca
58617,11214.0,10046.0,9042.0,7811.0,18375.0,11732.0,7570.0,-74.956899,1.188136,8554.0,9606.0,caqueta_bosque_20160311,bosque,bosque


In [78]:
#Buscando los datos de coca encontrados correctamente con SVM
targetCoca = metadata_df['Target'] == 'coca'
predictionCoca = metadata_df['Prediction'] == 'coca'
cocaOK = metadata_df[targetCoca & predictionCoca]
cocaOK.to_csv('./csv/cocaOK.csv', encoding='utf-8', index=False)
cocaOK.to_csv('./csv/back/cocaOK.csv', encoding='utf-8')
cocaOK

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target,Prediction
3,10628.0,9630.0,8682.0,7635.0,18300.0,11348.0,7559.0,-72.094163,2.220039,6844.0,2001.0,guaviare_20160125,coca,coca
4,11232.0,10146.0,9511.0,8343.0,18440.0,12152.0,8177.0,-75.335603,0.886033,5745.0,11832.0,caqueta_20160311,coca,coca
11,9836.0,8795.0,8080.0,6916.0,19394.0,12767.0,7907.0,-72.232831,2.206481,5715.0,2103.0,guaviare_20160820,coca,coca
15,11272.0,10247.0,9596.0,8580.0,19308.0,15406.0,10253.0,-75.155764,1.002754,7079.0,10972.0,caqueta_20160311,coca,coca
16,11069.0,9953.0,8967.0,7796.0,18844.0,11583.0,7494.0,-75.653009,0.089291,5670.0,7063.0,putumayo_20160311,coca,coca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16442,11106.0,10001.0,9075.0,7896.0,17735.0,11601.0,7628.0,-75.354764,1.055664,5603.0,10582.0,caqueta_20160311,coca,coca
16443,9684.0,8558.0,7744.0,6608.0,17699.0,11415.0,7252.0,-72.094383,1.953582,6746.0,3967.0,guaviare_20160820,coca,coca
16452,10590.0,9551.0,8644.0,7573.0,17652.0,11083.0,7485.0,-72.271548,2.105295,5529.0,2850.0,guaviare_20160125,coca,coca
16463,11547.0,10414.0,9300.0,8156.0,18322.0,11667.0,7619.0,-75.971799,0.362564,3305.0,5049.0,putumayo_20160311,coca,coca


In [79]:
#Buscando los datos de coca encontrados incorrectamente con SVM
targetCoca = metadata_df['Target'] == 'coca'
predictionCoca = metadata_df['Prediction'] != 'coca'
cocaFail = metadata_df[targetCoca & predictionCoca]
cocaFail.to_csv('./csv/cocaFail.csv', encoding='utf-8', index=False)
cocaFail.to_csv('./csv/back/cocaFail.csv', encoding='utf-8')
cocaFail

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target,Prediction
19,11734.0,10606.0,9587.0,8391.0,17650.0,10976.0,7526.0,-75.446786,0.463299,7200.0,4307.0,putumayo_20160311,coca,bosque
84,10728.0,9962.0,9576.0,9427.0,17842.0,15253.0,10236.0,-72.736521,1.979934,2080.0,3780.0,guaviare_20160125,coca,cana
119,10970.0,9933.0,9326.0,7915.0,25723.0,14862.0,9007.0,-75.308062,1.689014,5950.0,5915.0,caqueta_20160311,coca,bosque
231,9978.0,8867.0,7938.0,6873.0,18831.0,11465.0,7316.0,-72.326952,2.090344,5018.0,2961.0,guaviare_20160820,coca,bosque
241,9639.0,8607.0,8141.0,7013.0,19154.0,12395.0,7869.0,-71.992225,1.131332,7513.0,10033.0,guaviare_20160820,coca,cana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16314,9350.0,8293.0,7480.0,6412.0,16924.0,9447.0,6483.0,-71.576778,1.156336,10599.0,9845.0,guaviare_20160820,coca,bosque
16398,12464.0,11644.0,10920.0,10287.0,19591.0,16478.0,12509.0,-71.525898,2.162455,10964.0,2417.0,guaviare_20160820,coca,cana
16436,9580.0,8531.0,7675.0,6692.0,16807.0,10600.0,7138.0,-72.102866,1.408207,6689.0,7991.0,guaviare_20160820,coca,bosque
16471,10417.0,9364.0,8654.0,7521.0,19038.0,11056.0,7428.0,-71.994945,1.230672,7592.0,9300.0,guaviare_20160125,coca,bosque


In [80]:
#Buscando los datos de caña encontrados correctamente con SVM
targetCana = metadata_df['Target'] == 'cana'
predictionCana = metadata_df['Prediction'] == 'cana'
canaOK = metadata_df[targetCana & predictionCana]
canaOK.to_csv('./csv/canaOK.csv', encoding='utf-8', index=False)
canaOK.to_csv('./csv/back/canaOK.csv', encoding='utf-8')
canaOK

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target,Prediction
8,10434.0,9704.0,9607.0,9957.0,18112.0,17302.0,11598.0,-76.429210,3.266969,6688.0,4960.0,cauca_20190903,cana,cana
9,12030.0,11313.0,10557.0,10219.0,14417.0,11757.0,9613.0,-76.544211,3.087731,5834.0,6280.0,cauca_20190903,cana,cana
14,9157.0,8268.0,8383.0,7419.0,22859.0,11843.0,7799.0,-76.336794,3.198312,7372.0,5467.0,cauca_20190903,cana,cana
28,10556.0,9924.0,9873.0,9925.0,18518.0,19519.0,13268.0,-76.444053,3.168321,6577.0,5687.0,cauca_20190903,cana,cana
29,11669.0,11012.0,9446.0,9890.0,12694.0,10150.0,8493.0,-76.385182,3.044267,7012.0,6602.0,cauca_20190903,cana,cana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16474,9967.0,8994.0,8783.0,7389.0,19960.0,11031.0,7134.0,-76.391196,3.204752,7009.0,5419.0,cauca_20190818,cana,cana
16478,9735.0,8735.0,8786.0,6885.0,22764.0,11365.0,6996.0,-76.424991,3.144199,6758.0,5865.0,cauca_20190818,cana,cana
16480,12166.0,11423.0,11101.0,11260.0,18609.0,14952.0,11125.0,-76.363267,3.108873,7175.0,6126.0,cauca_20190903,cana,cana
16482,11044.0,10261.0,9843.0,9865.0,10628.0,14676.0,12482.0,-76.540574,3.092349,5901.0,6246.0,cauca_20190818,cana,cana


In [81]:
#Buscando los datos de caña encontrados incorrectamente con SVM
targetCana = metadata_df['Target'] == 'cana'
predictionCana = metadata_df['Prediction'] != 'cana'
canaFail = metadata_df[targetCana & predictionCana]
canaFail.to_csv('./csv/canaFail.csv', encoding='utf-8', index=False)
canaFail.to_csv('./csv/back/canaFail.csv', encoding='utf-8', index=False)
canaFail

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target,Prediction
13,10240.0,9402.0,8953.0,8483.0,18329.0,13872.0,9293.0,-76.400931,3.217627,6897.0,5324.0,cauca_20190903,cana,bosque
270,9989.0,9035.0,8397.0,7345.0,17278.0,10388.0,7421.0,-76.443542,3.188807,6621.0,5536.0,cauca_20190818,cana,bosque
441,10474.0,9597.0,8937.0,8714.0,13497.0,15639.0,11981.0,-76.442859,3.183245,6626.0,5577.0,cauca_20190818,cana,bosque
445,10002.0,9045.0,8128.0,7362.0,15597.0,12262.0,8018.0,-76.448810,3.192326,6582.0,5510.0,cauca_20190818,cana,bosque
483,10277.0,9494.0,9156.0,8665.0,18879.0,15833.0,10505.0,-76.428667,3.168342,6731.0,5687.0,cauca_20190818,cana,bosque
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15539,9979.0,9007.0,8747.0,7208.0,20318.0,11954.0,7916.0,-76.406692,3.186009,6894.0,5557.0,cauca_20190818,cana,bosque
15600,10645.0,9761.0,8935.0,7781.0,19135.0,11296.0,7837.0,-76.370423,3.214141,7123.0,5350.0,cauca_20190903,cana,bosque
15620,9943.0,8932.0,8545.0,6977.0,24887.0,10632.0,6918.0,-76.566232,3.104927,5671.0,6153.0,cauca_20190903,cana,bosque
15759,10712.0,9920.0,9545.0,8577.0,20759.0,14250.0,10313.0,-76.371648,3.119308,7113.0,6049.0,cauca_20190903,cana,bosque


In [82]:
#Buscando coca, clasificada correctamente por el clasificador y que pertenezcan a la imagen "Caqueta"
#para la fecha 20160311
img = cocaOK['AOI'] == 'caqueta_20160311'
cocaCaqueta = cocaOK[img]
cocaCaqueta

,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target,Prediction
4,11232.0,10146.0,9511.0,8343.0,18440.0,12152.0,8177.0,-75.335603,0.886033,5745.0,11832.0,caqueta_20160311,coca,coca
15,11272.0,10247.0,9596.0,8580.0,19308.0,15406.0,10253.0,-75.155764,1.002754,7079.0,10972.0,caqueta_20160311,coca,coca
24,11108.0,10022.0,9293.0,8040.0,20319.0,11890.0,7891.0,-75.415829,0.979797,5150.0,11141.0,caqueta_20160311,coca,coca
26,10954.0,9886.0,9220.0,7707.0,24535.0,12961.0,7868.0,-75.380687,1.323002,5411.0,8612.0,caqueta_20160311,coca,coca
39,11155.0,10114.0,9169.0,8088.0,16734.0,13141.0,9050.0,-75.447363,1.661574,4917.0,6117.0,caqueta_20160311,coca,coca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16402,11669.0,10594.0,10046.0,8676.0,21918.0,14765.0,9270.0,-75.130434,1.352613,7267.0,8394.0,caqueta_20160311,coca,coca
16422,11311.0,10241.0,9652.0,8404.0,18233.0,14557.0,9480.0,-75.345047,0.959585,5675.0,11290.0,caqueta_20160311,coca,coca
16431,11221.0,10062.0,9117.0,7977.0,16808.0,10986.0,7261.0,-75.306496,1.028935,5961.0,10779.0,caqueta_20160311,coca,coca
16442,11106.0,10001.0,9075.0,7896.0,17735.0,11601.0,7628.0,-75.354764,1.055664,5603.0,10582.0,caqueta_20160311,coca,coca


In [83]:
total_images_x = 7
total_images_y = 7
suma = 0
best = 0
i_img=0
j_img=0
for i in range(total_images_x):
    img_cond_x_min = cocaCaqueta['x']>i*2048
    img_cond_x_max = cocaCaqueta['x']<=(i+1)*2048
    for j in range(total_images_y):
        img_cond_y_min = cocaCaqueta['y']>j*2048
        img_cond_y_max = cocaCaqueta['y']<=(j+1)*2048
        cocaTmp = cocaCaqueta[img_cond_x_min & img_cond_x_max & img_cond_y_min & img_cond_y_max]
        suma += cocaTmp.shape[0]
        if(cocaTmp.shape[0]>best):
            best=cocaTmp.shape[0]
            i_img=i+1
            j_img=j+1
print("Best:", i_img, j_img, "With:", best)
img_cond_x_min = cocaCaqueta['x']>(i_img-1)*2048
img_cond_x_max = cocaCaqueta['x']<=(i_img)*2048
img_cond_y_min = cocaCaqueta['y']>(j_img-1)*2048
img_cond_y_max = cocaCaqueta['y']<=(j_img)*2048
cocaTmp = cocaCaqueta[img_cond_x_min & img_cond_x_max & img_cond_y_min & img_cond_y_max]
cocaTmp

Best: 4 5 With: 289


,Costal,Blue,Green,Red,NIR,SWIR1,SWIR2,Longitude,Latitude,x,y,AOI,Target,Prediction
149,11311.0,10239.0,9648.0,8376.0,21370.0,13595.0,8917.0,-75.277940,1.300488,6173.0,8778.0,caqueta_20160311,coca,coca
226,11308.0,10190.0,9435.0,7962.0,22270.0,12204.0,7733.0,-75.156727,1.356139,7072.0,8368.0,caqueta_20160311,coca,coca
242,11398.0,10352.0,9864.0,8504.0,21795.0,16143.0,9935.0,-75.228584,1.225854,6539.0,9328.0,caqueta_20160311,coca,coca
271,11407.0,10338.0,9703.0,8799.0,18225.0,14236.0,9441.0,-75.198382,1.225585,6763.0,9330.0,caqueta_20160311,coca,coca
303,11315.0,10183.0,9593.0,8178.0,22839.0,14244.0,8748.0,-75.248671,1.193961,6390.0,9563.0,caqueta_20160311,coca,coca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16199,11314.0,10159.0,9500.0,8189.0,19730.0,13780.0,8828.0,-75.137303,1.217446,7216.0,9390.0,caqueta_20160311,coca,coca
16234,11345.0,10222.0,9590.0,8144.0,22183.0,12752.0,8241.0,-75.275629,1.108734,6190.0,10191.0,caqueta_20160311,coca,coca
16271,11448.0,10376.0,9605.0,8697.0,16542.0,13800.0,9032.0,-75.135279,1.189898,7231.0,9593.0,caqueta_20160311,coca,coca
16304,11332.0,10343.0,9566.0,8996.0,16311.0,15871.0,10758.0,-75.256628,1.209431,6331.0,9449.0,caqueta_20160311,coca,coca


In [84]:
import h5py
import numpy as np

filename = '/media/ruben4181/Games/Caqueta/Pansharp_20160311/4_5.mat'
psh_img = None
with h5py.File('/media/ruben4181/Games/Caqueta/Pansharp_20160311/4_5.mat', 'r') as f:
    psh_img = f.get('I_BDSD')
    psh_img = np.array(psh_img).transpose()
psh_img.shape

(2048, 2048, 7)

In [85]:
#Función recursiva que busca si los pixeles aledaños contienen coca
import sys
sys.getrecursionlimit()
sys.setrecursionlimit(5000)
neighboors = {}
polygon = []
neighboors.keys()
def checkNeighboors(i, j):
    if i-1 >= 0 and j-1 >=0:
        if (i-1, j-1) not in neighboors.keys():
            s = psh_img[i-1][j-1]
            s = np.array(s).reshape(1, -1)
            if(classifier.predict(s)[0]=='coca'):
                polygon.append([i-1, j-1])
                neighboors[(i-1, j-1)]='coca'
                checkNeighboors(i-1, j-1)
    if i-1>=0:
        if(i-1, j) not in neighboors.keys():
            s = psh_img[i-1][j]
            s = np.array(s).reshape(1, -1)
            if(classifier.predict(s)[0]=='coca'):
                polygon.append([i-1, j])
                neighboors[(i-1, j)]='coca'
                checkNeighboors(i-1, j)
    if i-1>=0 and j+1<2048:
        if(i-1, j+1) not in neighboors.keys():
            s = psh_img[i-1][j+1]
            s = np.array(s).reshape(1, -1)
            if(classifier.predict(s)[0]=='coca'):
                polygon.append([i-1, j+1])
                neighboors[(i-1, j+1)]='coca'
                checkNeighboors(i-1, j+1)
    
    
u=0
master = []
for index, row in cocaTmp.iterrows():
    x_pos = int(row['x'])
    y_pos = int(row['y'])
    x_pos = x_pos%((i_img-1)*2048)-1
    y_pos = y_pos%((j_img-1)*2048)-1
    sample = psh_img[x_pos][y_pos]
    sample = np.array(sample).reshape(1, -1)
    polygon=[]
    if(classifier.predict(sample)[0]=='coca'):
        polygon.append([x_pos, y_pos])
        neighboors[(x_pos, y_pos)]='coca'
        checkNeighboors(x_pos, y_pos)
        print(len(polygon), '\n\n\n')
    if u==1:
        break
    else:
        u+=1

836 



105129 





In [86]:
s = psh_img[56][1745]
s = np.array(s).reshape(1, -1)
classifier.predict(s)[0]
print(s)

[[11440 10436  9804  8968 16920 15395 10353]]
